In [86]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_validate, ShuffleSplit, train_test_split
from sklearn.linear_model import LogisticRegressionCV
import pandas as pd
import data 
from collections import Counter

# plt.style.use('dark_background')
# sns.set(style="white")
%matplotlib inline
%load_ext autoreload
%autoreload 2
NUM_PATIENTS = 12044

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
features = data.get_features() # uses several forms
outcomes = data.get_outcomes() # 2 outcomes: iai, and iai_intervention
df = pd.merge(features, outcomes, on='id', how='left')
X_feats = data.preprocess(features)

# remove unusable features
ks = [k for k in X_feats.keys() if not k in ['id'] and not 'IAI' in k]
X_feats = X_feats[ks]

# get actual features
X_np = X_feats.values
feature_names = list(X_feats)
y_np = outcomes['iai_intervention']

100%|██████████| 49/49 [00:01<00:00, 39.79it/s]
48it [00:03, 14.01it/s]


final shape (12044, 433)


In [87]:
print(X_feats.shape, sorted(list(X_feats.keys())))

(12044, 109) ['AbdCTScan1_4a', 'AbdCTScan_1', 'AbdDistention_1_1', 'AbdDistention_1_2', 'AbdDistention_1_3', 'AbdDistention_1_4', 'AbdTenderDegree_1_1.0', 'AbdTenderDegree_1_2.0', 'AbdTenderDegree_1_3.0', 'AbdTenderDegree_1_4.0', 'AbdTenderDegree_1_nan', 'AbdTrauma_1_1', 'AbdTrauma_1_2', 'AbdTrauma_1_3', 'AbdTrauma_1_4', 'AbdTrauma_or_SeatBeltSign_1_False', 'AbdTrauma_or_SeatBeltSign_1_True', 'AbdUltrasound1_4a', 'AbdUltrasound_1', 'AbdomenPain_1_1', 'AbdomenPain_1_2', 'AbdomenPain_1_3', 'AbdomenPain_1_4', 'AbdomenTender_1', 'AbnChestAusc_1', 'AlcDrgSuspicion_1', 'BowelSounds_1', 'Certification_1', 'ChestCXR_4a', 'Costal_1_True', 'CurrIntubated_1', 'DecrBreathSound_1_1', 'DecrBreathSound_1_2', 'DecrBreathSound_1_3', 'DecrBreathSound_1_4', 'DistractingPain_1_1', 'DistractingPain_1_2', 'DistractingPain_1_3', 'DistractingPain_1_4', 'EDTriageTime_1', 'FlankTender_1', 'GCSEye_1', 'GCSMotor_1', 'GCSScore_1', 'GCSVerbal_1', 'HISPANIC_ETHNICITY_-1', 'HISPANIC_ETHNICITY_0', 'HISPANIC_ETHNICITY_

In [19]:
Counter(y_np)

Counter({0: 11841, 1: 203})

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.2)

In [34]:
Counter(y_train)

Counter({0: 9459, 1: 176})

# decision tree fitting

In [ ]:
model_type='logistic'
max_depth = 2
num_cv = 3

if model_type == 'tree':
    m = DecisionTreeClassifier(max_depth=max_depth, class_weight={0: 1, 1: 300})
elif model_type == 'logistic':
    m = LogisticRegressionCV(class_weight={0: 1, 1: 300}, cv=3, max_iter=100)
cv = ShuffleSplit(n_splits=10, test_size=0.1, random_state=42)
m_cv = cross_validate(m, X_train, y_train, cv=num_cv, scoring=['precision', 'recall', 'f1', 'balanced_accuracy', 'roc_auc'],
                      return_train_score=True, return_estimator=True)

In [93]:
# print('metric\ttrain\ttest')
for key in m_cv:
    if 'test' in key:
#         print(f"{key.replace('test_', '')}\t{np.mean(m_cv[key.replace('test', 'train')]):0.2f}\t{np.mean(m_cv[key]):0.2f}")
        print(f"{np.mean(m_cv[key]):0.2f}\t{key.replace('test_', '')}")

0.09	precision
0.87	recall
0.16	f1
0.85	balanced_accuracy
0.93	roc_auc


[('AbdCTScan1_4a', 3),
 ('RefuseFU_1', 3),
 ('OthAbdImages_4a', 3),
 ('PtHosp_1', 2),
 ('AbdTenderDegree_1_3.0', 2),
 ('AbdUltrasound1_4a', 1),
 ('AbdTenderDegree_1_nan', 1)]

In [ ]:
# plot a tree
m_fit = m_cv['estimator'][0]
plt.figure(dpi=300)
plot_tree(m_fit, feature_names=feature_names)
plt.show()